In [5]:
import os
import random
import sys
import glob
import librosa
import soundfile
import musdb
import numpy as np
import tensorflow as tf
from lxml import etree
from multiprocessing import Process

sys.path.append('../')
import Utils

In [18]:
main_folder = "../data/test_dataset"
musdb_path = '/Users/zhao/Downloads/MUS-STEMS-SAMPLE/'

mus = musdb.DB(root_dir=musdb_path, is_wav=False)
subsets = list()



for subset in ["train", "test"]:
    tracks = mus.load_mus_tracks(subset)
    samples = list()

    # Go through tracks
    for track in tracks:
        # Skip track if mixture is already written, assuming this track is done already
        track_path = track.path[:-4]
        mix_path = track_path + "_mix.wav"
        acc_path = track_path + "_accompaniment.wav"
        rate = track.rate
        print(rate)

        # Go through each instrument
        paths = dict()
        stem_audio = dict()
        for stem in ["bass", "drums", "other", "vocals"]:
            path_orig = track_path + "_" + stem + ".wav"
            path = track_path+"_"+stem+"_down.wav"
            audio = track.targets[stem].audio
            audio = downsample_mono_audio(audio)
            soundfile.write(path, audio, 16000, "PCM_16")
            stem_audio[stem] = audio
            paths[stem] = path

        # Add other instruments to form accompaniment
        acc_audio = np.clip(sum([stem_audio[key] for key in stem_audio.keys() if key != "vocals"]), -1.0, 1.0)
        soundfile.write(acc_path, acc_audio, 16000, "PCM_16")
        paths["accompaniment"] = acc_path

        # Create mixture
        mix_audio = downsample_mono_audio(track.audio)
        soundfile.write(mix_path, mix_audio, 16000, "PCM_16")
        paths["mix"] = mix_path

        diff_signal = np.abs(mix_audio - acc_audio - stem_audio["vocals"])
        print("Maximum absolute deviation from source additivity constraint: " + str(np.max(diff_signal)))# Check if acc+vocals=mix
        print("Mean absolute deviation from source additivity constraint:    " + str(np.mean(diff_signal)))

        samples.append(paths)

    subsets.append(samples)

subsets

44100
Maximum absolute deviation from source additivity constraint: 0.015788201498365628
Mean absolute deviation from source additivity constraint:    0.0016906991795110386
44100
Maximum absolute deviation from source additivity constraint: 1.0003329092154158
Mean absolute deviation from source additivity constraint:    0.020354763273284886


[[{'accompaniment': '/Users/zhao/Downloads/MUS-STEMS-SAMPLE/train/Music Delta - Rock.stem_accompaniment.wav',
   'bass': '/Users/zhao/Downloads/MUS-STEMS-SAMPLE/train/Music Delta - Rock.stem_bass_down.wav',
   'drums': '/Users/zhao/Downloads/MUS-STEMS-SAMPLE/train/Music Delta - Rock.stem_drums_down.wav',
   'mix': '/Users/zhao/Downloads/MUS-STEMS-SAMPLE/train/Music Delta - Rock.stem_mix.wav',
   'other': '/Users/zhao/Downloads/MUS-STEMS-SAMPLE/train/Music Delta - Rock.stem_other_down.wav',
   'vocals': '/Users/zhao/Downloads/MUS-STEMS-SAMPLE/train/Music Delta - Rock.stem_vocals_down.wav'}],
 [{'accompaniment': '/Users/zhao/Downloads/MUS-STEMS-SAMPLE/test/PR - Oh No.stem_accompaniment.wav',
   'bass': '/Users/zhao/Downloads/MUS-STEMS-SAMPLE/test/PR - Oh No.stem_bass_down.wav',
   'drums': '/Users/zhao/Downloads/MUS-STEMS-SAMPLE/test/PR - Oh No.stem_drums_down.wav',
   'mix': '/Users/zhao/Downloads/MUS-STEMS-SAMPLE/test/PR - Oh No.stem_mix.wav',
   'other': '/Users/zhao/Downloads/MUS-STE